In [1]:
from src.datasets.snli import *

In [2]:
snli_dataset_conf = {"batch_size":128, "device":'cuda','max_len':40,"tokenizer":"bert"}
data = snli(snli_dataset_conf)


In [2]:
from transformers import BertTokenizer, DistilBertTokenizer

In [3]:
tok = BertTokenizer.from_pretrained('bert-base-uncased')

In [1]:
import os
import sys

import torch
from transformers import BertTokenizer, DistilBertTokenizer

from torchtext.data import Field, Iterator, TabularDataset, BucketIterator, LabelField
from torchtext import datasets


In [11]:
import os
import sys

import torch
from transformers import BertTokenizer, DistilBertTokenizer

from torchtext.data import Field, Iterator, TabularDataset, BucketIterator, LabelField
from torchtext import datasets


options = {"batch_size":128, "device":'cuda','max_len':40,"tokenizer":"bert"}

if options["tokenizer"] == "bert":
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    def tokenize_bert(batch):
        return tokenizer.encode(batch,add_special_tokens=True,max_length=options['max_len'],padding='max_length')


    def tokenize_and_cut(sentence):
        tokens = tokenizer.tokenize(sentence) 
        tokens = tokens[:40-2]
        return tokens

    sepcial_tokens = tokenizer.special_tokens_map
    options["init_token"] = sepcial_tokens["cls_token"]
    options["pad_token"] = sepcial_tokens["pad_token"]
    options["unk_token"] = sepcial_tokens["unk_token"]
    options["eos_token"] = sepcial_tokens["sep_token"]
    options["use_vocab"] = False
    options["tokenize"] = tokenize_and_cut
    options["tokenizer"] = tokenizer

if options["tokenizer"] == "spacy":
        options["tokenize"] = "spacy"
        options["init_token"] = "<sos>"
        options["unk_token"] = "<unk>"
        options["pad_token"] = "<pad>"
        options["eos_token"] = "<eos>"
        options["use_vocab"] = True

if options.get("lower", None) == None:
    options["lower"] = True

 
TEXT = Field(
            preprocessing = options["tokenizer"].convert_tokens_to_ids,
            batch_first=True,
            use_vocab=options["use_vocab"],
            tokenize = options["tokenize"],
            init_token=options["init_token"],
            eos_token=options["eos_token"],
            pad_token=options["pad_token"],
            unk_token=options["unk_token"],
        )
LABEL = LabelField(dtype = torch.float)
train, dev, test = datasets.SNLI.splits(TEXT, LABEL)

LABEL.build_vocab(train)

In [12]:
train_iter, dev_iter, test_iter = BucketIterator.splits(
            (train, dev, test),
            batch_size=options["batch_size"],
            device=options["device"])

In [26]:
from torchtext.data import LabelField

In [14]:
print(train_iter.data()[0].label)

neutral


In [5]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f79a413a488>, {'entailment': 0, 'contradiction': 1, 'neutral': 2})


In [16]:
train.examples[0].label

'neutral'

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
len(tokenizer.vocab)


30522

In [4]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


In [5]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


In [6]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [7]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [8]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [9]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [10]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [11]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [12]:
from torchtext import datasets
import random

train_data, valid_data, test_data = datasets.SNLI.splits(TEXT, LABEL)


In [13]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 549367
Number of validation examples: 9842
Number of testing examples: 9824


In [14]:
print(vars(train_data.examples[6]))

{'premise': [1037, 2879, 2003, 8660, 2006, 17260, 6277, 1999, 1996, 2690, 1997, 1037, 2417, 2958, 1012], 'hypothesis': [1996, 2879, 17260, 2015, 2091, 1996, 11996, 1012], 'label': 'contradiction'}


In [16]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['premise'])

print(tokens)

['a', 'boy', 'is', 'jumping', 'on', 'skate', '##board', 'in', 'the', 'middle', 'of', 'a', 'red', 'bridge', '.']


In [17]:
LABEL.build_vocab(train_data)

In [18]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f68b8d9a378>, {'entailment': 0, 'contradiction': 1, 'neutral': 2})


In [19]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [20]:
for i in train_iterator:
    print(i)
    break


[torchtext.data.batch.Batch of size 128 from SNLI]
	[.premise]:[torch.cuda.LongTensor of size 128x32 (GPU 0)]
	[.hypothesis]:[torch.cuda.LongTensor of size 128x22 (GPU 0)]
	[.label]:[torch.cuda.FloatTensor of size 128 (GPU 0)]
